In this notebook I will implement a long short term memory (LSTM) neural network to predict the closing stock price of a corporation (Apple) using the past 60 day stock price.

In [1]:
# Import libraries
import math
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')